In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [20]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [21]:
df.industry.mode()

0    retail
Name: industry, dtype: object

In [22]:
df.select_dtypes('number').corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.031551,-0.023565,-0.004879,0.435914
annual_income,0.031551,1.000000,0.048618,0.005334,0.078256
interaction_count,-0.023565,0.048618,1.000000,0.009888,0.374573
lead_score,-0.004879,0.005334,0.009888,1.000000,0.193673
converted,0.435914,0.078256,0.374573,0.193673,1.000000


interaction_count and lead_score: 0.009888
number_of_courses_viewed and lead_score: -0.004879
number_of_courses_viewed and interaction_count: -0.023565
annual_income and interaction_count: 0.048618

In [23]:
def data_split(df, test_size, val_size, random_state=42):
	df, df_test = train_test_split(df, test_size=test_size, random_state=random_state)
	df_train, df_val = train_test_split(df, test_size=val_size/(1-test_size), random_state=random_state)

	return df_train, df_val, df_test

In [24]:
df_train, df_val, df_test = data_split(df, test_size=.2, val_size=.2, random_state=42)

In [25]:
df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1077,paid_ads,retail,0,58472.0,student,middle_east,5,0.03,0
463,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77,1
842,paid_ads,technology,3,81973.0,employed,north_america,2,0.59,1
835,NaN,technology,1,74956.0,employed,europe,3,0.34,1
837,organic_search,retail,3,59335.0,student,australia,1,0.98,1


In [26]:
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [27]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

df_train.drop('converted', axis=1, inplace=True)
df_val.drop('converted', axis=1, inplace=True)
df_test.drop('converted', axis=1, inplace=True)